In [1]:
import numpy as np
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Model, Reaction, Metabolite
from os.path import join

data_dir = "/Users/phivri/Documents/GitHub/Biosustain/MoGeoModel"

WLModel = cobra.io.read_sbml_model(join(data_dir,'Models/WLPathway_bifurcating.xml'))

Using license file /Users/phivri/gurobi.lic
Academic license - for non-commercial use only - expires 2022-07-23


In [2]:
medium_carb = WLModel.medium
medium_carb['EX_co_e'] = 1000
medium_carb['EX_co2_e'] = 0
medium_carb['EX_h2_e'] = 0

medium_hom = WLModel.medium
medium_hom['EX_co_e'] = 0
medium_hom['EX_co2_e'] = 1000
medium_hom['EX_h2_e'] = 1000

In [3]:
WLModel.objective = "EX_ac_e"

with WLModel:
    WLModel.medium = medium_carb
    print("Carboxydotrophic acetate production flux: ", WLModel.slim_optimize(), "\n")
    Acetate_ex_carb = WLModel.slim_optimize()
    CO2_exc_carb = WLModel.reactions.EX_co2_e.flux/Acetate_ex_carb
    H2_exc_carb = WLModel.reactions.EX_h2_e.flux/Acetate_ex_carb
    CO_exc_carb = WLModel.reactions.EX_co_e.flux/Acetate_ex_carb
    H2O_exc_carb = WLModel.reactions.EX_h2o_e.flux/Acetate_ex_carb

with WLModel:
    WLModel.medium = medium_hom
    print("Homoacetogenic acetate production flux: ", WLModel.slim_optimize(), "\n")
    Acetate_ex_hom = WLModel.slim_optimize()
    CO2_exc_hom = WLModel.reactions.EX_co2_e.flux/Acetate_ex_carb
    H2_exc_hom = WLModel.reactions.EX_h2_e.flux/Acetate_ex_carb
    CO_exc_hom = WLModel.reactions.EX_co_e.flux/Acetate_ex_carb
    H2O_exc_hom = WLModel.reactions.EX_h2o_e.flux/Acetate_ex_carb
    
print(f"Carboxydotroph: {-CO_exc_carb} CO + {round(-H2O_exc_carb,2)} H2O => {CO2_exc_carb} CO2 + {1} acetate")
print(f"Homoacetogen: {-CO2_exc_hom} CO2 + {-H2_exc_hom} H2 => {round(H2O_exc_hom,2)} H2O + {1} acetate")

Y_IE_carb = -1/CO_exc_carb*2
Y_IE_hom = -1/H2_exc_hom*2
print(f"Carboxydotrophic acetate yield: {Y_IE_carb} [Cmol/emol]")
print(f"Homoacetogenic acetate yield: {Y_IE_hom} [Cmol/emol]")

Carboxydotrophic acetate production flux:  1.0 

Homoacetogenic acetate production flux:  1.0 

Carboxydotroph: 4.0 CO + -0.0 H2O => 2.0 CO2 + 1 acetate
Homoacetogen: 2.0 CO2 + 4.0 H2 => 2.67 H2O + 1 acetate
Carboxydotrophic acetate yield: 0.5 [Cmol/emol]
Homoacetogenic acetate yield: 0.5 [Cmol/emol]


# Add Heterotroph

In [4]:
#Bsubt = cobra.io.read_sbml_model(join(data_dir,'Models/iJO1366.xml'))
Geob = cobra.io.read_sbml_model(join(data_dir,'Models/p-thermo_acetone_anaerobic.xml'))

OSError: [Errno The file with 'filename' does not exist, or is not an SBML string. Provide the path to an existing SBML file or a valid SBML string representation: 
%s] /Users/phivri/Documents/GitHub/Biosustain/MoGeoModel/Models/p-thermo_acetone_anaerobic.xml

Change objective function and remove constraints for ATP uptake for maintenance. This allows to ignore both GAM and NGAM

In [ ]:
Geob.objective = "EX_act_e" #don't force growth

Geob.reactions.ATPM.bounds = 0,1000 #Ignore NGAM
Geob.reactions.GAPDH.bounds = 0,1000 #Futile cycle discovered, due to error in bounds
Geob.reactions.ATPM.bounds

In [ ]:
medium = Geob.medium
medium["EX_glc__D_e"] = 0
medium["EX_o2_e"] = 1000
medium["EX_ac_e"] = 1
Geob.medium = medium

solution = cobra.flux_analysis.pfba(Geob)
Y_PI = Geob.slim_optimize()*3/(1*2)

print("Heterotrophic acetone yield on acetate: ", round(Y_PI,2), "[Cmol/Cmol]")

Geob.summary()

In [ ]:
Builder(model = Geob, map_json = join(data_dir,'Models/EscherMaps/P-thermo_AcetateDegrad.json'), reaction_data = solution.fluxes.to_dict())

More or less in accordance with:  10.3390/ijms21228777
Check and compare critically though

In [ ]:
print(f"Succesive carboxydotrophic acetone yield: {round(Y_IE_carb*Y_PI,2)} [Cmol/emol]")
print(f"Succesive homoacetogenic acetone yield: {round(Y_IE_hom*Y_PI,2)} [Cmol/emol]")

In [ ]:
Builder(model = Geob, map_json = join(data_dir,'Models/EscherMaps/iJO1366.CentralMetabolism.json'), reaction_data = solution.fluxes.to_dict())

In [ ]:
Builder(model = Geob, map_json = join(data_dir,'Models/EscherMaps/AcetonePathway2.json'), reaction_data = solution.fluxes.to_dict())